In [ ]:
# Tutorial: Geometric Analysis of the Embodied Telepresence Connection (ETC)

**Objective:** To quantify the "shape" of inter-brain synchronization during pseudo-haptic interactions using Forman-Ricci Curvature (FRC) and Network Entropy (RNE).

**Prerequisites:**

* **Python 3.8+**

* **Libraries:** `numpy`, `pandas`, `scipy`, `networkx`, `matplotlib`, `statsmodels`

* **Data:** \* Preprocessed fNIRS time-series (LIONirs output with `NaNs`)

  * VR Kinematic logs (Head rotation, Hand displacement)

  * ECG logs (Sec-by-sec PNS/Stress index)

## Part 1: Dual-fNIRS Preprocessing (The LIONirs Handler)

The LIONirs pipeline replaces motion artifacts with `NaNs`. For geometric analysis, we need continuous phase data. We must impute short gaps while respecting the "Invalid" status of long gaps.

### 1.1 Dyadic Loading & Imputation Strategy

We process both subjects simultaneously to ensure time-series lengths match exactly before any correlation is calculated.

```python
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.stats import zscore

def rigorous_imputation(series, sampling_rate=10, max_gap_sec=2.0):
    """
    Interpolates NaN gaps < 2s using Cubic Spline.
    Returns: cleaned_series, valid_mask (False if gap > 2s)
    """
    ts = series.copy()
    max_gap_samples = int(max_gap_sec * sampling_rate)
    is_nan = np.isnan(ts)

    # Group consecutive NaNs
    groups = is_nan.astype(int).groupby(is_nan.astype(int).diff().ne(0).cumsum()).cumsum()
    valid_mask = np.ones(len(ts), dtype=bool)

    for group_id, group in groups.groupby(groups):
        if is_nan.iloc[group.index[0]]: # Found a NaN block
            if len(group) <= max_gap_samples:
                # Interpolate
                start, end = group.index[0] - 1, group.index[-1] + 1
                if start >= 0 and end < len(ts):
                    x = [start, end]
                    y = [ts.iloc[start], ts.iloc[end]]
                    cs = CubicSpline(x, y)
                    ts.iloc[group.index] = cs(group.index)
            else:
                # Mark invalid
                valid_mask[group.index] = False

    return ts, valid_mask

def process_dyad_fnirs(df_s1, df_s2, channels):
    """
    Applies imputation to paired fNIRS data.
    """
    dyad_data = {}
    validity_masks = []

    for ch in channels:
        # Subject 1
        s1_clean, m1 = rigorous_imputation(df_s1[ch])
        # Subject 2
        s2_clean, m2 = rigorous_imputation(df_s2[ch])

        dyad_data[f"S1_{ch}"] = zscore(s1_clean, nan_policy='omit')
        dyad_data[f"S2_{ch}"] = zscore(s2_clean, nan_policy='omit')

        # Global validity: If either S1 or S2 is invalid, the pair is invalid
        validity_masks.append(m1 & m2)

    # Combine masks: strict intersection (data must be valid in ALL channels to compute topology)
    global_mask = np.logical_and.reduce(validity_masks)

    return pd.DataFrame(dyad_data), global_mask

## Part 2: VR Kinematics Processing

**Challenge:** VR data often logs at variable frame rates (e.g., 90Hz) or simply "on event". We need to resample this to match the fNIRS sampling rate (approx 10Hz) and calculate "Motion Energy" to regress out of the neural signal.

**Data Source:** `Kinematic data structure-2.pdf` (Head Rotation X, Y, Z).

### 2.1 Resampling & Motion Energy Calculation

In [ ]:
def process_kinematics(vr_file_path, target_timestamps):
    """
    Loads VR logs, extracts head rotation, and resamples to fNIRS timestamps.
    """
    df_vr = pd.read_csv(vr_file_path) # Assuming CSV export

    # 1. Create a datetime index or relative time index for VR
    # conversion of 'Time' column to seconds if needed

    # 2. Extract Head Rotation columns
    # Adjust column names based on actual file header
    cols_of_interest = [
        'P1 head rotation x', 'P1 head rotation y', 'P1 head rotation z',
        'P2 head rotation x', 'P2 head rotation y', 'P2 head rotation z',
        'P1 hand displacement', 'P2 hand displacement'
    ]

    # 3. Resample to fNIRS target_timestamps (linear interpolation)
    vr_resampled = pd.DataFrame(index=target_timestamps)

    for col in cols_of_interest:
        # Interpolate VR data onto fNIRS timepoints
        vr_resampled[col] = np.interp(
            target_timestamps,
            df_vr['Time'], # VR time column
            df_vr[col]
        )

    # 4. Calculate Composite Motion Energy (Derivative)
    # We want the change in rotation (velocity) as the artifact source
    for p in ['P1', 'P2']:
        rot_cols = [f'{p} head rotation {ax}' for ax in ['x', 'y', 'z']]
        # Euclidean norm of the derivative
        diffs = vr_resampled[rot_cols].diff().fillna(0)
        vr_resampled[f'{p}_Head_Motion_Energy'] = np.sqrt(
            diffs.iloc[:,0]**2 + diffs.iloc[:,1]**2 + diffs.iloc[:,2]**2
        )

    return vr_resampled

### 2.2 Neural Regression

We remove variance in HbO caused purely by head movement.

In [ ]:
import statsmodels.api as sm

def regress_out_motion(neural_signal, motion_regressors):
    """
    neural_signal: 1D array (fNIRS channel)
    motion_regressors: 2D array (Head Motion Energy X, Y, Z)
    """
    X = sm.add_constant(motion_regressors)
    model = sm.OLS(neural_signal, X, missing='drop').fit()
    return model.resid # The "cleaned" neural signal

## Part 3: ECG Data Processing

**Challenge:** ECG data (`ECG data structure.pdf`) is "sec-by-sec". If fNIRS is 10Hz, we have a mismatch.
**Goal:** Align PNS Index (Vagal Tone) to use as a predictor for Geometric Entropy.

### 3.1 Alignment Strategy

Since `PNS Index` changes slowly (autonomic state), we step-interpolate (forward fill) it to match the fNIRS 10Hz resolution.

In [ ]:
def process_ecg(ecg_file_path, target_timestamps):
    df_ecg = pd.read_csv(ecg_file_path)

    # Columns from PDF
    target_cols = ['PNS index', 'Stress index', 'Mean HR']

    ecg_aligned = pd.DataFrame(index=target_timestamps)

    for col in target_cols:
        # Use 'nearest' or 'previous' interpolation for physiological states
        # fNIRS time 10.1s should take ECG value from 10.0s
        f = CubicSpline(df_ecg['Time'], df_ecg[col])
        ecg_aligned[col] = f(target_timestamps)

    return ecg_aligned

## Part 4: Constructing Annotated Weighted Connectivity Matrices

This step fuses the neural, physiological, and behavioral data. We iterate through time windows, creating a snapshot of the brain network (Weighted Matrix) and tagging it with the concurrent ECG and VR states (Annotations).

In [ ]:
def get_condition_info(timestamp):
    """
    Placeholder: Map a timestamp to your experimental block (Touch, ISI, Free).
    You should implement the specific logic from your 'ETC_dual person_short.pptx' timings here.
    """
    # Example logic:
    # if 0 < timestamp < 60: return "Pseudo", "Free"
    # if 60 < timestamp < 78: return "Pseudo", "ISI"
    return "Unknown", "Unknown"

def build_annotated_graphs(fnirs_dyad, vr_data, ecg_data, window_size=150, step_size=10):
    """
    Generates a sequence of weighted connectivity matrices annotated with physiological and behavioral states.

    Args:
        fnirs_dyad (pd.DataFrame): Cleaned, z-scored fNIRS data (columns: S1_Ch1, ..., S2_Ch1...)
        vr_data (pd.DataFrame): Resampled VR kinematics (Head motion, Hand displacement).
        ecg_data (pd.DataFrame): Aligned ECG metrics (PNS index).
        window_size (int): Samples per window (e.g., 15s * 10Hz = 150).
        step_size (int): Samples to shift (e.g., 1s * 10Hz = 10).

    Returns:
        list: A list of dictionary objects, where each object represents one time window.
    """
    annotated_graphs = []

    # 1. Define loop parameters
    num_samples = len(fnirs_dyad)

    # 2. Sliding Window Loop
    for start_idx in range(0, num_samples - window_size, step_size):
        end_idx = start_idx + window_size

        # --- A. Slice the Data Streams ---
        window_fnirs = fnirs_dyad.iloc[start_idx:end_idx]
        window_vr = vr_data.iloc[start_idx:end_idx]
        window_ecg = ecg_data.iloc[start_idx:end_idx]

        # --- B. Quality Check ---
        # If too many NaNs remain (from valid_mask in Part 1), skip this window
        if window_fnirs.isnull().mean().mean() > 0.1:
            continue

        # --- C. Build Connectivity Matrix (The "Network") ---
        # Weighted Pearson Correlation: W_ij = |r_ij|
        corr_matrix = window_fnirs.corr(method='pearson')

        # Convert to absolute weights (standard for functional connectivity)
        weight_matrix = corr_matrix.abs().values

        # Zero out self-loops (diagonal)
        np.fill_diagonal(weight_matrix, 0)

        # --- D. Aggregate Annotations (The "Context") ---
        # Calculate the mean state of the participants *during this specific network snapshot*

        # Get experimental phase info
        current_time = window_fnirs.index[0]
        condition, phase = get_condition_info(current_time)

        annotation = {
            'timestamp_start': current_time,
            'timestamp_end': window_fnirs.index[-1],
            'condition': condition,
            'phase': phase,

            # Physiological Annotations
            'mean_PNS': window_ecg['PNS index'].mean(),
            'mean_Stress': window_ecg['Stress index'].mean(),

            # Behavioral Annotations (Force/Embodiment proxy)
            'mean_hand_displacement': window_vr[[c for c in window_vr.columns if 'hand displacement' in c]].mean().mean(),

            # Control Variable (Motion Artifact proxy)
            'mean_head_motion': window_vr[[c for c in window_vr.columns if 'Motion_Energy' in c]].mean().mean()
        }

        # --- E. Store ---
        annotated_graphs.append({
            'matrix': weight_matrix,   # This goes to the Geometric Toolkit
            'metadata': annotation     # This goes to the Statistical Model
        })

    return annotated_graphs

## Part 5: Geometric Computation (The HyPhi Pipeline)

With our list of `annotated_graphs`, we can now compute the topology.

### 5.1 Augmented Forman-Ricci Curvature (AFRC)

In [ ]:
import networkx as nx

def compute_afrc(adj_matrix):
    G = nx.from_numpy_array(adj_matrix)
    curvature_dict = {}

    for u, v in G.edges():
        w_uv = G[u][v]['weight']
        if w_uv == 0: continue

        # Node weights ~ sum of neighbors
        s_u = sum([G[u][n]['weight'] for n in G[u] if n != v])
        s_v = sum([G[v][n]['weight'] for n in G[v] if n != u])

        # Triangles (Cycles)
        triangles = sorted(list(nx.common_neighbors(G, u, v)))
        triangle_sum = 0
        for t in triangles:
            w_ut = G[u][t]['weight']
            w_vt = G[v][t]['weight']
            triangle_sum += max(w_ut, w_vt)

        # AFRC Formula Approximation
        # F(e) ~ 4 - degrees + triangles
        frc = w_uv * (2 - (s_u/np.sqrt(w_uv) + s_v/np.sqrt(w_uv)) + triangle_sum)
        curvature_dict[(u,v)] = frc

    return list(curvature_dict.values())

### 5.2 Ricci Network Entropy (RNE) & Final Integration

We iterate through the `annotated_graphs` list, compute the RNE for each matrix, and flatten the result into a DataFrame for statistics.

In [ ]:
from scipy.stats import entropy

def compute_rne(curvature_values, bins=20):
    counts, _ = np.histogram(curvature_values, bins=bins, density=True)
    return entropy(counts + 1e-10)

def run_geometric_pipeline(annotated_graphs):
    results = []

    for item in annotated_graphs:
        matrix = item['matrix']
        meta = item['metadata']

        # 1. Compute Curvature
        curvatures = compute_afrc(matrix)

        # 2. Compute Entropy
        rne = compute_rne(curvatures)

        # 3. Combine for export
        row = meta.copy()
        row['RNE'] = rne
        row['Mean_Curvature'] = np.mean(curvatures)
        results.append(row)

    return pd.DataFrame(results)

# Usage:
# final_df = run_geometric_pipeline(annotated_graphs)
# final_df.to_csv("ETC_Geometric_Analysis_Results.csv")

## Part 6: Statistical Evaluation

**Final Model Structure:**

We construct a master DataFrame where every row is one time-window (t).

| **Time** | **Condition** | **Phase** | **RNE (Outcome)** | **PNS_Index** | **Hand_Disp** |
| 15.0 | Pseudo | Touch | 2.45 | \-0.5 | 0.02 |
| 16.0 | Pseudo | Touch | 2.89 | \-0.4 | 0.15 |

**Hypothesis Testing (LMM):**
`RNE ~ Condition * Phase + Motion_Energy + PNS_Index + (1 | DyadID)`

* **Higher RNE** in Pseudo-Touch implies the brain is geometrically reorganizing to solve the sensory conflict.

* **PNS Index** interaction: Does high vagal tone (safety) allow for higher geometric flexibility (higher RNE)?
```
  ```